# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bsmith53\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bsmith53\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [4]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

A subdirectory or file data already exists.


In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 31506    0 31506    0     0   168k      0 --:--:-- --:--:-- --:--:--  171k


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 70532    0 70532    0     0   498k      0 --:--:-- --:--:-- --:--:--  514k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg


Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 10, relationships: 24)

We can save and load our knowledge graphs as follows.

In [15]:
import locale
print(locale.getpreferredencoding())


cp65001


In [16]:
import sys

print("Default encoding:", sys.getdefaultencoding())         # usually 'utf-8'
print("Filesystem encoding:", sys.getfilesystemencoding())   # often 'utf-8' or 'mbcs' (Windows)




Default encoding: utf-8
Filesystem encoding: utf-8


In [17]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 10, relationships: 24)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [19]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [20]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is Mistral in the context of large langua...,[The ethics of this space remain diabolically ...,Mistral is mentioned as one of the organizatio...,single_hop_specifc_query_synthesizer
1,"What are Large Language Models, and why are th...",[Simon Willison’s Weblog Subscribe Stuff we fi...,Large Language Models are the latest and most ...,single_hop_specifc_query_synthesizer
2,"As an AI research enthusiast, how does the doc...",[the document includes some of the clearest ex...,The document mentions Stanford Alpaca in the c...,single_hop_specifc_query_synthesizer
3,"As an AI research enthusiast, could you explai...",[The rise of inference-scaling “reasoning” mod...,Reka AI is mentioned among the organizations w...,single_hop_specifc_query_synthesizer
4,What is the significance of Nova in the contex...,[you talk to me exclusively in Spanish. OpenAI...,Amazon pre-announced voice mode for Amazon Nov...,single_hop_specifc_query_synthesizer
5,How do data reqirements and trainng proses for...,[<1-hop>\n\nThe ethics of this space remain di...,The context explains that the most important f...,multi_hop_abstract_query_synthesizer
6,so like Simon Willison says 2023 was big year ...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"According to Simon Willison’s weblog, 2023 was...",multi_hop_abstract_query_synthesizer
7,H0w does the enviromental impact of AI trainng...,[<1-hop>\n\nThe rise of inference-scaling “rea...,The context explains that the environmental im...,multi_hop_abstract_query_synthesizer
8,"Gemini and also Gemini like, what they do with...",[<1-hop>\n\nyou talk to me exclusively in Span...,Google’s Gemini accepts audio input and has a ...,multi_hop_specific_query_synthesizer
9,Whay is Claude and how does it relate to Gemin...,[<1-hop>\n\nyou talk to me exclusively in Span...,Claude is an AI model developed by Anthropic t...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [21]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [22]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Is Stability AI a major player in LLM developm...,[The ethics of this space remain diabolically ...,"Yes, according to the context, Stability AI is...",single_hop_specifc_query_synthesizer
1,What is the role of Microsoft in the developme...,"[and software engineer, LLMs are infuriating. ...",The provided context does not specify the role...,single_hop_specifc_query_synthesizer
2,Considering the developments in AI during 2023...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, it was considered the breakthrough ye...",single_hop_specifc_query_synthesizer
3,what is stanford alpaca,[the document includes some of the clearest ex...,The context mentions Stanford Alpaca in relati...,single_hop_specifc_query_synthesizer
4,How do recent advancements in multimodal capab...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Recent advancements in multimodal capabilities...,multi_hop_abstract_query_synthesizer
5,LLMs smart or dumb how about Simon Willison say?,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,Simon Willison’s weblog says 2023 was a big ye...,multi_hop_abstract_query_synthesizer
6,Considering the various organizations developi...,[<1-hop>\n\nThe ethics of this space remain di...,The context highlights that many organizations...,multi_hop_abstract_query_synthesizer
7,"How do the advancements in LLM capabilities, s...",[<1-hop>\n\nThe rise of inference-scaling “rea...,The context segments highlight significant adv...,multi_hop_abstract_query_synthesizer
8,china why did the inference scaling models in ...,[<1-hop>\n\nThe rise of inference-scaling “rea...,"The context indicates that in 2024, models tra...",multi_hop_specific_query_synthesizer
9,How do recent developments in multi-modal audi...,[<1-hop>\n\nthe document includes some of the ...,The context highlights that ChatGPT has recent...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [23]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [24]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [25]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [28]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

c:\Users\bsmith53\AIE6_NEW\AIE6\07_Synthetic_Data_Generation_and_LangSmith\.venv\Lib\site-packages\qdrant_client\http\models\models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
c:\Users\bsmith53\AIE6_NEW\AIE6\07_Synthetic_Data_Generation_and_LangSmith\.venv\Lib\site-packages\qdrant_client\http\models\models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [30]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [33]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an extremely vague and loosely defined term in AI. It generally refers to AI systems that can act on your behalf, similar to a travel agent or a digital assistant, often involving LLMs (Large Language Models) given access to tools which they can use iteratively to solve problems. However, there is no single clear or widely understood meaning of "agents," and despite much discussion and excitement, true AI agents capable of autonomous meaningful decisions have not yet been realized in production. The concept often involves autonomy but lacks a precise definition, and the term still carries the sense of something "coming soon" but not yet fully achieved. Additionally, the challenge of "gullibility" (LLMs believing anything told to them) remains a major obstacle in developing effective agents.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [34]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dope_or_nope_evaluator`:

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'ordinary-tax-32' at:
https://smith.langchain.com/o/bec07d7d-df09-4fc3-917e-46e6c637d955/datasets/ca714c29-70f8-4c4f-8791-4e4d6bb51c72/compare?selectedSessions=6647c627-f942-4221-9617-3d30d0fee0ca




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do recent developments in LLM training dat...,Recent developments in LLM training data and l...,None,Recent developments highlight that the most su...,1,1,0,5.205680,16e322d0-8f3a-4e7b-bd98-0e0229a7c0ee,771dc8a9-3b98-4196-a835-bfdd7c14333d
1,How do synthetic data and ChatGPT exemplify re...,"Based on the provided context, synthetic train...",None,The context highlights that synthetic data is ...,1,1,0,2.963796,71c2e20d-5813-4254-83cb-85779350a2c7,85962382-7dd1-477e-9142-5a92bad2481d
2,How do recent developments in multi-modal audi...,Recent developments in multi-modal audio and v...,None,The context highlights that ChatGPT has recent...,1,1,0,5.697341,c5463e7d-560d-4c0f-bf60-e01f058c9dc7,bf0cb3e4-c32e-407e-a6ba-1d5acb1bbb71
3,china why did the inference scaling models in ...,Based on the context provided:\n\nInference-sc...,None,"The context indicates that in 2024, models tra...",1,1,0,6.176016,c3cc7bc8-d8bd-4d39-b26e-8227f4f7a2da,e3da354f-a317-4474-8a3f-2900d4782a3a
4,"How do the advancements in LLM capabilities, s...","Advancements in LLM capabilities, particularly...",None,The context segments highlight significant adv...,1,1,0,4.122528,3d4f81e9-178b-4391-a88c-3858e8ec4d7f,2a24468d-32b1-4e4a-a345-f3a103209a7b
5,Considering the various organizations developi...,"Based on the provided context, several insight...",None,The context highlights that many organizations...,1,1,0,8.418399,87b81c9c-5d19-4217-9a27-7e17fa06974d,c7db6e07-3ae3-42f3-adc9-7e59344c5749
6,LLMs smart or dumb how about Simon Willison say?,"According to Simon Willison, LLMs are ""really ...",None,Simon Willison’s weblog says 2023 was a big ye...,1,1,0,1.057563,9819fe6e-5dc6-46bb-8ee1-2c44d5a5676a,459acd5a-e2ce-4b97-9e80-7fa7d51beff8
7,How do recent advancements in multimodal capab...,Recent advancements in multimodal capabilities...,None,Recent advancements in multimodal capabilities...,1,1,0,11.497102,ebd6eeea-5d49-4c2a-970f-36702cbd97aa,2a0a1865-ff44-49b0-941f-6ae1b66ca078
8,what is stanford alpaca,I don't know,None,The context mentions Stanford Alpaca in relati...,0,0,0,0.734940,9203d35b-ada4-4f52-b74f-0452edc48ed7,f72954e4-2d28-4a82-a634-88b3037f87cc
9,Considering the developments in AI during 2023...,Based on the provided context from Simon Willi...,None,"In 2023, it was considered the breakthrough ye...",1,1,0,7.235808,8e2f0f79-f184-48ef-af11-73af92d82d22,c53f6c62-1875-4d28-b7f8-7b915f0eed9d


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [37]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [38]:
rag_documents = docs

In [39]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [40]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [41]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [42]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [43]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [44]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, listen up! Agents, in the AI game, are kinda like these elusive digital sidekicks folks keep hyping but nobody’s nailed down a solid, universally accepted definition yet. Some peeps think agents are AI that go and act on your behalf—like your personal travel agent but for anything. Others see them as Large Language Models (LLMs) equipped with tools they can loop through to solve problems. But here’s the kicker: the whole “autonomy” vibe gets tossed around with no clear playbook.\n\nThe real tea? Agents are still in that “coming soon” phase. The big obstacle? Gullibility. These AI agents believe anything you feed them, and until we crack that code—possibly needing AGI (Artificial General Intelligence) to really shine—the dream of trustworthy agents acting independently is kinda on pause. So yeah, agents are these promised AI helpers, but right now, they’re more concept than reality, struggling with separating fact from fiction and dodging prompt trickery.\n\nIn a nutshell: Age

Finally, we can evaluate the new chain on the same test set!

In [45]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'pertinent-relation-14' at:
https://smith.langchain.com/o/bec07d7d-df09-4fc3-917e-46e6c637d955/datasets/ca714c29-70f8-4c4f-8791-4e4d6bb51c72/compare?selectedSessions=687a9ef9-1dff-4905-8188-5e691184bf75




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How do recent developments in LLM training dat...,"Alright, here's the lowdown, fresh and fly fro...",None,Recent developments highlight that the most su...,1,1,1,5.889800,16e322d0-8f3a-4e7b-bd98-0e0229a7c0ee,83d767b9-179a-44b0-bcf8-57231031b1d3
1,How do synthetic data and ChatGPT exemplify re...,"Alright, listen up! Synthetic data and ChatGPT...",None,The context highlights that synthetic data is ...,1,0,1,4.362749,71c2e20d-5813-4254-83cb-85779350a2c7,1a92fd17-e09a-451f-891c-2b952350673f
2,How do recent developments in multi-modal audi...,"Oh, you wanna know how those fresh audio and v...",None,The context highlights that ChatGPT has recent...,1,1,1,8.617033,c5463e7d-560d-4c0f-bf60-e01f058c9dc7,14058b53-c19c-40d0-89e4-a36d363a5e5e
3,china why did the inference scaling models in ...,"Alright, here’s the lowdown: China’s inference...",None,"The context indicates that in 2024, models tra...",1,1,1,4.150263,c3cc7bc8-d8bd-4d39-b26e-8227f4f7a2da,a7b0baff-e2e8-4320-ab8e-cef5d420ca25
4,"How do the advancements in LLM capabilities, s...","Alright, here’s the lowdown, straight from the...",None,The context segments highlight significant adv...,1,1,1,4.150854,3d4f81e9-178b-4391-a88c-3858e8ec4d7f,bc298c94-6e5a-4d28-a1be-44147603fb95
5,Considering the various organizations developi...,"Alright, here’s the lowdown, straight from the...",None,The context highlights that many organizations...,1,1,1,5.527776,87b81c9c-5d19-4217-9a27-7e17fa06974d,18d42a38-14db-45e4-90fc-cc101df0f451
6,LLMs smart or dumb how about Simon Willison say?,"Yo, Simon Willison’s take is straight-up real ...",None,Simon Willison’s weblog says 2023 was a big ye...,1,1,1,1.886403,9819fe6e-5dc6-46bb-8ee1-2c44d5a5676a,b13741df-1cc0-49f4-a287-14f795103f6c
7,How do recent advancements in multimodal capab...,"Yo, here’s the lowdown on those fresh multimod...",None,Recent advancements in multimodal capabilities...,1,1,1,3.639428,ebd6eeea-5d49-4c2a-970f-36702cbd97aa,4855f11a-a58d-47d3-8d30-8c313d1b1c05
8,what is stanford alpaca,"Yo, based on the vibe from the context, Stanfo...",None,The context mentions Stanford Alpaca in relati...,1,1,1,1.946302,9203d35b-ada4-4f52-b74f-0452edc48ed7,ae0a67eb-5ac9-4073-bce7-6ba385536ce0
9,Considering the developments in AI during 2023...,"Yo, check this out—2023 was the ultimate glow-...",None,"In 2023, it was considered the breakthrough ye...",1,1,1,4.184536,8e2f0f79-f184-48ef-af11-73af92d82d22,9a300ba9-74a7-4c22-be50-e35872938cb3


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.